In [21]:
%load_ext autoreload
%autoreload 2

In [43]:
import numpy as np
import matplotlib.pyplot as plt
import knn as kalmann
from load import load_bikes_data
from tqdm import tqdm 
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from  tqdm import tqdm

##  Load Data

In [44]:
X,y = load_bikes_data()
print(y.shape)
print(X.shape)

(17379,)
(17379, 14)


## Preprocess Data

In [45]:
scaler = MinMaxScaler()

X_scaled = scaler.fit_transform(X)

## Partition Dataset

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled,y,test_size=.5)
X_val,X_test, y_val,y_test = train_test_split(X_test,y_test,test_size=0.5)
print(f'Train Shape: {X_train.shape}')
print(f'Validation Shape: {X_val.shape}')
print(f'Test shape: {X_test.shape}')

Train Shape: (8689, 14)
Validation Shape: (4345, 14)
Test shape: (4345, 14)


## Create EKF network 

In [47]:
n_inputs = X_train.shape[1]

knn_ekf = kalmann.KNN(nu=n_inputs, ny=1, nl=10, neuron='relu')


In [50]:
nepochs = 100
knn_ekf.train(nepochs=100, U=X_train, Y=y_train, U_val=X_val,Y_val=y_val, method='ekf', P=100, Q=10e-6, R=10, print_every=10)
print("Done")

Training...


  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

------------------
  Epoch: 1
   RMSE: 266.468737
tr(Cov): 4313.281174
------------------


 10%|████████                                                                         | 10/100 [00:23<03:30,  2.34s/it]

------------------
  Epoch: 11
   RMSE: 0.00225
tr(Cov): 4360.538373
------------------


...

------------------
  Epoch: 81
   RMSE: 0.0
tr(Cov): 5220.980022
------------------


 90%|████████████████████████████████████████████████████████████████████████▉        | 90/100 [03:33<00:23,  2.32s/it]

------------------
  Epoch: 91
   RMSE: 0.0
tr(Cov): 5344.377004
------------------


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [03:56<00:00,  2.21s/it]



Training complete!


Done


In [51]:
y_pred= knn_ekf.feedforward(X_test)
error = mean_squared_error(y_test,y_pred)
np.sqrt(error)

0.0016003582877578252